# Consistent Bayes: Some Motivating Examples
---

Copyright 2017-2018 Michael Pilosov


### Import Libraries
_tested on 3.6 01/23/18_

In [1]:
# Mathematics and Plotting
from HelperFuns import * # pyplot wrapper functions useful for visualizations, numpy, scipy, etc.
%matplotlib inline
plt.rcParams.update({'font.size': 14})
plt.rcParams['figure.figsize'] = 10, 5
from cbayes import sample, solve
# Interactivity
from ipywidgets import *

---

Some introductory text goes here.   
Define $\Lambda$, $\mathcal{D}$.


--- 

## Sample from $\Lambda$
_Here we implement uniform random priors on the unit hypercube, but you can load in any set of samples in its place._

In [2]:
input_dim = 2 # Specify input space dimension (n)
num_samples = int(1E4) # number of input samples (N)
s_set = sample.sample_set(size=(num_samples, input_dim))
s_set.set_dist('uniform')
s_set.generate_samples()
lam = s_set.samples

---
## Define Parameter to Observables (PtO) Map
_ Choose from one of the following example options, feel free to add your own _ 

$O_1(\lambda) = \sum_{i=1}^n \lambda_i$  

$O_2(\lambda) = \alpha \, \lambda_1$, $\alpha = 2$  

$O_3(\lambda) = \lbrace \lambda_0 - \lambda_1, \; \;\lambda_1\rbrace$ 

$O_4(\lambda) = \lbrace \lambda_0+\lambda_1, \; \lambda_2, \; \lambda_3-\lambda_4 \rbrace$  

In [3]:
def fun1(lam): # sum all params
    return np.sum(lam,axis=1)

def fun2(lam): # pull param, scale it.
    return np.array([ 2*lam[:,0] ]).transpose()

def fun3(lam): # pull two params, linear combination.
    return np.array([ lam[:,0] - lam[:,1], lam[:,1] ]).transpose()

def fun4(lam): # pull three sets of params, linear combinations thereof.
    return np.array([ lam[:,0]+lam[:,1], lam[:,2], lam[:,3]-lam[:,4] ]).transpose()

In [4]:
PtO_fun_choice = 1

if PtO_fun_choice == 1:
    PtO_fun = fun1
elif PtO_fun_choice == 2:
    PtO_fun = fun2
elif PtO_fun_choice == 3:
    PtO_fun = fun3
elif PtO_fun_choice == 4:
    PtO_fun = fun4
else:
    raise( ValueError('Specify Proper PtO choice!') )

---
## Compute Data Space $O(\Lambda) = \mathcal{D}$ 

Format: `(n_dims, n_samples)`  
_Optional_: Specify subset of PtO map's components to use for inversion using the variable `sub_indices` 

In [5]:
p_set = sample.map_samples_and_create_problem(s_set, PtO_fun)
D_full = p_set.output.samples

sub_indices = None
if sub_indices is not None:
    D = D_full[:,sub_indices]
else:
    D = D_full
    type(np.sum(lam, axis=1))
# this is how we handle exceptions:
try:
    output_dim = D.shape[1]
except IndexError:
    output_dim = 1
print('dimensions :  lambda = '+str(lam.shape)+'   D = '+str(D.shape)+'   D_full = '+str(D_full.shape) )

dimensions :  lambda = (10000, 2)   D = (10000, 1)   D_full = (10000, 1)


## Compute Push-Forward of the Prior $P_{O(\Lambda)}$
_ ... i.e. Visualize the Data Space_

In [6]:
# Interactive Marginal Visualization
p_set.compute_pushforward_dist()
pf_dens = p_set.pushforward_dist

# Can plot "slices" of densities to observe differences between posterior and prior, but not that useful

a, b = -0.25, 2.125 # linspace parameters for plotting
plot_grid = np.linspace(a, b, 100)

widgets.interact(view_est_dens, x = fixed(plot_grid), 
         estimated_dens = fixed(pf_dens), 
         lab = fixed('KDE data'), title=fixed('Pushforward of Prior'),
         viewdim=(0, output_dim-1, 1) )
plt.show()

interactive(children=(IntSlider(value=0, description='viewdim', max=0), Output()), _dom_classes=('widget-interact',))

## Define Observed Probability Measure $P_\mathcal{D}$

In [7]:
# We will use multivariate normals because of their common scipy.stats call syntaxes.
var_const = 0.25 # constant variance for simplicity
cov_matrix = var_const*np.eye(output_dim) # if you want non-constant, use code snippet below
# cov_matrix = np.diag( np.round(np.random.rand(1,output_dim),2)[0] ) # random independent covariances
means = np.zeros(output_dim) + 0.875 # observed density center (maximum)
# obs_dens = sstats.multivariate_normal(mean = means, cov = cov_matrix )
p_set.set_observed_dist('uniform', means, var_const)
obs_dens = p_set.observed_dist

# # For the 1-dimensional case, we can utilize the uniform density given below
# # TODO: generalize uniform box to n-dimensional case
# uni_max = 0.5
# obs_dens = sstats.uniform(0,uni_max) # 1D only

# a, b = 0, 1 # linspace parameters for plotting
plot_grid = np.linspace(a, b, 100)

widgets.interact(view_analytical_dens, x = fixed(plot_grid), 
         analytical_dens = fixed(obs_dens), 
         lab = fixed('KDE data'), title=fixed('Observed $P_\mathcal{D}$'),
         viewdim=(0, output_dim-1, 1) )
plt.show()

interactive(children=(IntSlider(value=0, description='viewdim', max=0), Output()), _dom_classes=('widget-interact',))

---

At this point we have performed the computations we need to. We have evaluated the input points through our map and performed a KDE on them. It would be useful at this point to save this object and/or its evaluation at every point in the data space for later re-use. Doing so here would be an appropriate place. 

--- 

## Accept/Reject Sampling of Posterior

Since we have already used the samples in our prior to compute the pushforward density, we can re-use these with an accept/reject algorithm to get a set of samples generated from the posterior according to the solution of the stochastic inverse problem as outlined in the Consistent Bayes formulation. 

In [12]:
p_set.set_ratio()
eta_r = p_set.ratio
solve.problem(p_set)

In [15]:
accept_inds = p_set.accept_inds
lam_accept = p_set.input.samples[accept_inds,:]
num_accept = len(accept_inds)
print('Number accepted: %d = %2.2f%%'%(num_accept, 100*np.float(num_accept)/num_samples))

Number accepted: 2173 = 21.73%


### Visualize Accept/Reject Samples

In [14]:
interact(pltaccept, lam = fixed(lam), inds = fixed(accept_inds), 
         N = (1, num_accept+1, 10), eta_r = fixed(eta_r), 
         i = (0, input_dim-1, 1), j = (0, input_dim-1, 1))
# You will visualize the accepted samples in a subset of size N of the input samples
plt.show()

interactive(children=(IntSlider(value=1081, description='N', max=2174, min=1, step=10), IntSlider(value=0, description='i', max=1), IntSlider(value=1, description='j', max=1), Output()), _dom_classes=('widget-interact',))

---
## Visualize Posterior Density

In [17]:
prior_dens_kde = gauss_kde(lam.transpose()) # this method wants dimensions different than ours.
post_dens_kde = gauss_kde(lam_accept.transpose())

a, b = 0, 1 # linspace parameters for plotting
plot_grid = np.linspace(a, b, 100)

# Can plot "slices" of densities to observe differences between posterior and prior, but not that useful
interact(compare_est_input_dens, x = fixed(plot_grid), 
         estimated_dens1 = fixed(prior_dens_kde), estimated_dens2 = fixed(post_dens_kde), 
         lab_1 = fixed('KDE prior'), lab_2 = fixed('KDE post'), title=fixed(''),
         viewdim=(0, input_dim-1, 1))
plt.show()

interactive(children=(IntSlider(value=0, description='viewdim', max=1), Output()), _dom_classes=('widget-interact',))

## Visualize Quality of Solution 
_We compare the push-forward of the posterior using accepted samples against the observed density_

In [23]:
D[accept_inds,:]

array([[1.06954891],
       [1.0880222 ],
       [0.93651026],
       ...,
       [0.99381507],
       [1.06552197],
       [0.98601326]])

In [20]:
push_post_dens_kde = gauss_kde(np.array([D[accept_inds,k] for k in range(output_dim)]).transpose())

a, b = 0, 1 # linspace parameters for plotting
plot_grid = np.linspace(a, b, 100)


# diagonal crossection view
interact(compare_output_dens, x = fixed(plot_grid), 
         analytical_dens = fixed(obs_dens), estimated_dens = fixed(push_post_dens_kde), 
         lab_1 = fixed('observed'), lab_2 = fixed('KDE push'), title = fixed('Marginal - Observed v. PF'),
         viewdim = (0, output_dim-1, 1))
plt.show()

/home/mpilosov/anaconda3/envs/test_py_env/lib/python3.6/site-packages/scipy-1.0.0-py3.6-linux-x86_64.egg/scipy/stats/kde.py:509: RuntimeWarning: Degrees of freedom <= 0 for slice
  bias=False))
/home/mpilosov/anaconda3/envs/test_py_env/lib/python3.6/site-packages/numpy-1.14.0-py3.6-linux-x86_64.egg/numpy/lib/function_base.py:3109: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)
/home/mpilosov/anaconda3/envs/test_py_env/lib/python3.6/site-packages/numpy-1.14.0-py3.6-linux-x86_64.egg/numpy/lib/function_base.py:3109: RuntimeWarning: invalid value encountered in multiply
  c *= 1. / np.float64(fact)


ValueError: array must not contain infs or NaNs